## **Given Task 3**

3. **Task Decomposition and Workflow Management**

   - Develop at least three specialised agent types with different capabilities (e.g., research agent, coding agent, writing agent).
   - Organise them is a multi-agent squad where workIntegrate at least two external tools or APIs that agents can use to complete tasks (e.g., web search API, code execution environment).
   - Implement a Retrieval-Augmented Generation (RAG) system to provide relevant
   - context to agents during task execution.

In [1]:
%%capture

!pip install -r requirements.txt

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
api_base_secret = os.getenv('api_base_secret')
azure_api_key_secret = os.getenv('azure_api_key_secret')

In [4]:
config_list = [{
    "engine": "gpt-35-turbo-16k", 
    "model": "gpt-35-turbo-16k",
    "api_base": api_base_secret, # Your Secret Base URL Endpoint :)
    "api_type": "azure", 
    "api_version": "2023-07-01-preview",
    "api_key": azure_api_key_secret # Your Secret Azure API key :)
}]

In [5]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
import chromadb
import autogen
# from tqdm.autonotebook import tqdm, trange


llm_config = {
    "request_timeout": 120,
    "seed": 21,
    "config_list": config_list,
    "temperature": 0.1,
}

autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

project_lead = autogen.UserProxyAgent(
    name="project_lead",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The Lead who ask questions and give tasks.",
    code_execution_config=False,  # we don't want to execute code in this case.
)


research_agent = RetrieveUserProxyAgent(
    name="research_agent",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult research problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/facebook/react/main/README.md",
        "chunk_token_size": 1500,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="owais_react_developer/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

coding_agent = autogen.UserProxyAgent(
    name="coding_agent",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The person who is very well versed in Coding and programmings.",
    code_execution_config=False,  # we don't want to execute code in this case.
)


writing_agent = AssistantAgent(
    name="writing_agent",
    is_termination_msg=termination_msg,
    system_message="Act as a agent who is a ghost writer, knows how to write fresh energitic content ",
    llm_config=llm_config,
)

PROBLEM = "How to build a simple React UI. Give me sample code and ways to execute it in console terminal"

def _reset_agents():
    research_agent.reset()
    coding_agent.reset()
    writing_agent.reset()
    project_lead.reset()

In [6]:
def rag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[research_agent, coding_agent, writing_agent], messages=[], max_round=10
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with research_agent as this is the user proxy agent.
    research_agent.initiate_chat(
        manager,
        problem=PROBLEM,
        n_results=5,)
    
    
rag_chat()

Trying to create collection.


/Users/owaisahmad/Desktop/Agentic Worker/env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/owaisahmad/Desktop/Agentic Worker/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


doc_ids:  [['doc_0']]
Adding doc_id doc_0 to context.
research_agent (to chat_manager):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How to build a simple React UI. Give me sample code and ways to execute it in console terminal

Context is: # [React](https://react.dev/) &middot; [![GitHub license](https://img.shields.io/badge/license-MIT-blue.svg)](https://github.com/facebook/react/blob/main/LICENSE) [![npm version](https://img.shields.io/npm/v/react.svg?style=flat)](https://www.npmjs.com/package/react) [![(Runtime) Build

In [7]:
def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[research_agent, coding_agent, writing_agent], messages=[], max_round=10
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with project_lead as this is the user proxy agent.
    project_lead.initiate_chat(manager,message=PROBLEM,)
norag_chat()

project_lead (to chat_manager):

How to build a simple React UI. Give me sample code and ways to execute it in console terminal

--------------------------------------------------------------------------------
coding_agent (to chat_manager):



--------------------------------------------------------------------------------
research_agent (to chat_manager):



--------------------------------------------------------------------------------
coding_agent (to chat_manager):



--------------------------------------------------------------------------------
research_agent (to chat_manager):



--------------------------------------------------------------------------------
writing_agent (to chat_manager):

As a ghostwriter, I can provide you with a sample code and steps to build a simple React UI. Here's an example of a basic React component:

```jsx
import React from 'react';

class SimpleUI extends React.Component {
  constructor(props) {
    super(props);
    this.state = {
      messag

project_lead (to chat_manager):

Monitor agent performance, visualize metrics, and propose optimizations.

--------------------------------------------------------------------------------
monitoring_agent (to chat_manager):

As the project lead, your role in monitoring agent performance, visualizing metrics, and proposing optimizations is crucial for the success of the project. Here are some key responsibilities you should focus on:

1. Define performance metrics: Work with stakeholders to identify key performance indicators (KPIs) that align with project goals. These metrics could include task completion rates, customer satisfaction scores, response times, or any other relevant measures.

2. Implement monitoring systems: Set up systems to collect and track performance data. This could involve using tools like performance management software, CRM systems, or custom-built dashboards. Ensure that the data is accurate, reliable, and easily accessible.

3. Regularly review performance metr